# Script 1 - S_D

Runs different models for data which are:

- Target: difference in scores


In [1]:
import pandas as pd
import numpy as np

import os
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
%%time
cols = ['Assets', 'Actions', 'Competency',
       'Form_record', 'Form_template', 'Incident', 'Users']

CentralStatistics = pd.read_csv('Central_Statistics.csv')

for drop in ['A', '-26']:
    for nweeks in range(1, 13):
        for r in ['r', 'r2', '1']:
            for fsval in [0.2, 0.3, 0.4]:
                for fold in range(1, 6):

                    # Training
                    models = dict()

                    for attb in cols:   
                        df = pd.read_csv(f'./S_D_SplitData/S_D_{drop}_{attb}_{nweeks}_Train_{fold}.csv')

                        x = df[df.columns[4:-1]]
                        y = df[df.columns[-1]]

                        lm = linear_model.LinearRegression()
                        model = lm.fit(x, y)

                        if lm.score(x, y) > fsval:
                            models[attb] = (model, lm.score(x, y))

                    if not models:
                        CentralStatisticCol = {'TargetType': ['S'], 'TargetMeth': ['D'], 'Method': [drop], 'NWEEKS': [nweeks],
                             'r': [r], 'fsval': [fsval], 'Fold': [fold], 'TP': [None], 'BFP': [None], 'TP1': [None], 
                             'TP2': [None], 'BFP1': [None], 'BFP2': [None]}
                        CentralStatistics = CentralStatistics.append(pd.DataFrame(CentralStatisticCol))
                        continue

                    # Testing
                    test = pd.read_csv(f'./S_D_SplitData/S_D_{drop}_{attb}_{nweeks}_Test_{fold}.csv')
                    out = test[test.columns[0:4]]
                    
                    # Predicting
                    for col in models.keys():
                        test = pd.read_csv(f'./S_D_SplitData/S_D_{drop}_{attb}_{nweeks}_Test_{fold}.csv')

                        xTest = test[test.columns[4:-1]]
                        yTest = test[test.columns[-1]]

                        lm = models[col][0]

                        yPred = lm.predict(xTest)

                        out[f'{col}O'] = yTest 
                        out[f'{col}P'] = yPred

                    out['ObsScore'] = [0 for i in range(len(out))]
                    out['PredScore'] = [0 for i in range(len(out))]

                    if r == 'r':
                        for col in models.keys():
                            out['ObsScore'] = out['ObsScore'] + models[col][1]*out[f'{col}O']
                            out['PredScore'] = out['PredScore'] + models[col][1]*out[f'{col}P']

                    elif r == 'r2':
                        for col in models.keys():
                            out['ObsScore'] = out['ObsScore'] + ((models[col][1])**2)*out[f'{col}O']
                            out['PredScore'] = out['PredScore'] + ((models[col][1])**2)*out[f'{col}P']
                    else:
                        for col in models.keys():
                            out['ObsScore'] = out['ObsScore'] + out[f'{col}O']
                            out['PredScore'] = out['PredScore'] + out[f'{col}P']

                    final = pd.DataFrame()
                    
                    # Labelling as increase, decrease or normal
                    for week, data in out.groupby('Week'):
                        data.index = range(len(data))

                        O95 = np.quantile(data['ObsScore'], .95)
                        O05 = np.quantile(data['ObsScore'], .05)

                        P95 = np.quantile(data['PredScore'], .95)
                        P05 = np.quantile(data['PredScore'], .05)

                        data['Obs'] = ['Increase' if data.loc[i]['ObsScore'] > O95 else 'Decrease' if data.loc[i]['ObsScore'] < O05 else 'Normal' for i in range(len(data))]
                        data['Pred'] = ['Increase' if data.loc[i]['PredScore'] > P95 else 'Decrease' if data.loc[i]['PredScore'] < P05 else 'Normal' for i in range(len(data))]

                        final = pd.concat([final, data])
                    
                    # Calculating true pos true neg statistics
                    n1 = len(final[final['Pred'] == 'Increase'])
                    n2 = len(final[final['Pred'] == 'Decrease'])
                    
                    o1 = len(final[final['Obs'] == 'Increase'])
                    o2 = len(final[final['Obs'] == 'Decrease'])
                    
                    tp1 = len(final[(final['Pred'] == 'Increase') & (final['Obs'] == final['Pred'])])/n1
                    tp2 = len(final[(final['Pred'] == 'Decrease') & (final['Obs'] == final['Pred'])])/n2
                    bfp1 = len(final[(final['Obs'] == 'Decrease') & (final['Pred'] == 'Increase')])/n1
                    bfp2 = len(final[(final['Obs'] == 'Increase') & (final['Pred'] == 'Decrease')])/n2
                    
                    s1 = len(final[(final['Obs'] == 'Increase') & (final['Obs'] == final['Pred'])])/o1
                    s2 = len(final[(final['Obs'] == 'Decrease') & (final['Obs'] == final['Pred'])])/o2


                    CentralStatisticCol = {'TargetType': ['S'], 'TargetMeth': ['D'], 'Method': [drop], 'NWEEKS': [nweeks],
                             'r': [r], 'fsval': [fsval], 'Fold': [fold], 'TP': [(tp1 + tp2)/2], 'BFP': [(bfp1+bfp2)/2], 
                            'Sens': [(s1+s2)/2], 'TP1': [tp1], 'TP2': [tp2], 'BFP1': [bfp1], 'BFP2': [bfp2], 'Sens1': [s1], 'Sens2': [s2]}

                    CentralStatistics = CentralStatistics.append(pd.DataFrame(CentralStatisticCol))

                CentralStatistics.to_csv('S_D_.csv', index = None)
            

C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand